In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import psycopg2 as pg2

In [3]:
conn = pg2.connect(dbname='nhl', user='postgres', host='localhost', port='5435')
cur = conn.cursor()

In [11]:
query = '''
SELECT * 
FROM pst;
'''

In [12]:
cur.execute(query)

In [10]:
for row in cur:
    print(row)

(4124, 103, 'Roberto Luongo', 'Goaltender', 12, 5333333, 63999996, 9.39, datetime.date(2009, 9, 2), 2009, 30, None, 'UFA', 'UFA')
(2837, 155, 'Ryan Miller', 'Goaltender', 3, 6000000, 18000000, 8.7, datetime.date(2014, 7, 1), 2014, 33, 'Vancouver Canucks', 'UFA', 'UFA')
(2832, 155, 'Ryan Miller', 'Goaltender', 2, 2000000, 4000000, 2.67, datetime.date(2017, 7, 1), 2017, 36, 'Anaheim Ducks', 'UFA', 'UFA')
(4078, 156, 'Craig Anderson', 'Goaltender', 3, 4200000, 12600000, 6.09, datetime.date(2014, 8, 25), 2014, 33, None, 'UFA', 'UFA')
(4079, 156, 'Craig Anderson', 'Goaltender', 2, 4750000, 9500000, 6.33, datetime.date(2017, 9, 29), 2017, 36, None, 'UFA', 'UFA')


In [13]:
pst = list(cur)

In [15]:
len(pst)

9673

In [18]:
pst_cols = ['Season_Player', 'Player', 'Season', 'Position', 'GP', 'TOI', 'Goals', 
                'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
               'IPP', 'Shots', 'SH%', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
               'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
               'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
               'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost',
               'Faceoffs %'
                ]

In [19]:
df = pd.DataFrame(pst, columns = pst_cols)

In [20]:
df.head()

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %
0,2017 Connor McDavid,Connor McDavid,2017,C,82,1766.80,41,67,39,28,...,0,36,67,111,28,118,46,376,533,41.36
1,2017 Claude Giroux,Claude Giroux,2017,C,82,1669.80,34,68,33,35,...,0,26,44,34,30,54,23,638,450,58.64
2,2017 Nikita Kucherov,Nikita Kucherov,2017,R,80,1585.72,39,61,33,28,...,0,36,79,66,31,92,15,3,2,60.00
3,2017 Evgeni Malkin,Evgeni Malkin,2017,C,78,1481.22,42,56,32,24,...,1,30,73,75,48,98,32,460,592,43.73
4,2017 Nathan MacKinnon,Nathan MacKinnon,2017,C,74,1472.98,39,58,38,20,...,1,24,41,36,38,74,22,485,672,41.92


In [24]:
df.iloc[0]

Season_Player       2017 Connor McDavid
Player                   Connor McDavid
Season                             2017
Position                              C
GP                                   82
TOI                              1766.8
Goals                                41
Total Assists                        67
First Assists                        39
Second Assists                       28
Total Points                        108
IPP                               88.52
Shots                               275
SH%                               14.91
iCF                                 434
iFF                                 357
iSCF                                329
iHDCF                               173
Rush Attempts                        18
Rebounds Created                     34
PIM                                  14
Total Penalties                       7
Minor                                 7
Major                                 0
Misconduct                            0


In [21]:
(df.Goals / df.Shots)[0]

0.14909090909090908

In [22]:
df['SH%'][0]

14.91

In [43]:
#can't sum SH%, faceoff%, or IPP so we're ignoring them for now to make things simpler

In [27]:
dfsummable = df.drop(['SH%', 'Faceoffs %', 'IPP'], axis=1)

In [30]:
df['SH%'].mean()

7.388850408353132

In [40]:
df[df['Player'] == 'Alex Ovechkin'][['SH%', 'Season']]

,SH%,Season
10,13.80,2017
907,10.54,2016
1789,12.56,2015
2676,13.42,2014
3563,13.21,2013
4443,14.55,2012
5316,12.54,2011
6179,8.72,2010
7064,13.59,2009
7941,10.61,2008


In [42]:
dfsummable_wind = dfsummable.copy()
dfsummable_car = dfsummable.copy()

In [44]:
window = 3

In [46]:
summable_stats = ['GP', 'TOI', 'Goals', 
                'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
               'Shots', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
               'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
               'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
               'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost']

In [57]:
dfsummable.sample(10)

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Major,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost
1842,2015 Tomas Plekanec,Tomas Plekanec,2015,C,82,1519.6300,14,40,21,19,...,0,0,19,45,15,60,66,46,804,811
2147,2015 Cody Franson,Cody Franson,2015,D,59,993.4000,4,13,6,7,...,0,0,2,31,14,61,39,56,0,0
392,2017 Leo Komarov,Leo Komarov,2017,C,74,1168.2300,7,12,4,8,...,1,0,9,42,47,219,66,48,112,100
2520,2015 Michal Jordan,Michal Jordan,2015,D,36,545.9000,1,0,0,0,...,0,0,3,8,6,24,59,34,0,0
6096,2011 Pierre-Luc Letourneau-Leblond,Pierre-Luc Letourneau-Leblond,2011,R,3,14.5667,0,0,0,0,...,0,1,1,1,0,2,0,0,0,0
7659,2009 Jay Harrison,Jay Harrison,2009,D,38,559.2670,1,5,1,4,...,6,0,9,11,6,67,35,46,0,0
6563,2010 Jeff Tambellini,Jeff Tambellini,2010,L,62,730.5670,9,8,5,3,...,0,0,5,10,11,113,73,21,19,16
7329,2009 Sami Salo,Sami Salo,2009,D,68,1406.1700,9,19,7,12,...,0,0,9,29,16,43,50,60,0,0
6029,2011 Cody Bass,Cody Bass,2011,C,14,127.2170,0,1,1,0,...,6,0,10,4,4,22,14,7,5,3
6199,2010 Lubomir Visnovsky,Lubomir Visnovsky,2010,D,81,1967.7200,18,50,18,32,...,0,0,8,45,30,52,76,116,0,0


In [58]:
X = dfsummable_wind.sort_values(by=['Season'])
g = X.groupby('Player')['Total Points']

AttributeError: Cannot access attribute 'values' of 'SeriesGroupBy' objects, try using the 'apply' method

In [65]:
m = g.apply(lambda x: x.rolling(3))

In [66]:
m

Season              Rolling [window=3,center=False,axis=0]
GP                  Rolling [window=3,center=False,axis=0]
TOI                 Rolling [window=3,center=False,axis=0]
Goals               Rolling [window=3,center=False,axis=0]
Total Assists       Rolling [window=3,center=False,axis=0]
First Assists       Rolling [window=3,center=False,axis=0]
Second Assists      Rolling [window=3,center=False,axis=0]
Total Points        Rolling [window=3,center=False,axis=0]
Shots               Rolling [window=3,center=False,axis=0]
iCF                 Rolling [window=3,center=False,axis=0]
iFF                 Rolling [window=3,center=False,axis=0]
iSCF                Rolling [window=3,center=False,axis=0]
iHDCF               Rolling [window=3,center=False,axis=0]
Rush Attempts       Rolling [window=3,center=False,axis=0]
Rebounds Created    Rolling [window=3,center=False,axis=0]
PIM                 Rolling [window=3,center=False,axis=0]
Total Penalties     Rolling [window=3,center=False,axis=

In [67]:
X = dfsummable_wind.sort_values(by=['Season'])
g = X.groupby('Player')
g

In [68]:
contracts = pd.read_csv('../data/Contract_Details_PuckPedia_Mar_2019_Confidential.csv')

In [69]:
def clean_data(file = None):
    #load in dataframe from csv
    if file:
        df = pd.read_csv(file)
    else:
        df = pd.read_csv('../data/Contract_Details_PuckPedia_Mar_2019_Confidential.csv')

    #Drop columns I won't be using
    df.drop(['buyout_id', 'base_salary', 'p_bonuses', 's_bonuses',
            'total_salary'], axis=1, inplace=True)

    #rename Total Value column to a format I prefer
    df['total_value'] = df[' Total Value ']
    df.drop(' Total Value ', axis=1, inplace=True)

    #create a column for full player name
    df['Player'] = df['first_name'] + ' ' + df['last_name']

    #make a function to convert money strings to ints and apply
    money_to_int = lambda x: int(x.strip().strip('$()').replace(',', ''))
    df['cap_hit'] = df['cap_hit'].apply(money_to_int)
    df['total_value'] = df['total_value'].apply(money_to_int)

    #convert signing date and birthdate to date format and pull out the signing year
    df.signing_date = pd.to_datetime(df.signing_date)
    df.birthdate = pd.to_datetime(df.birthdate)
    df['signing_year'] = pd.DatetimeIndex(df.signing_date).year

    #get age at contract signing_date (not accounting for leap days)
    df['signing_age'] = df.signing_date - df.birthdate
    df.signing_age = df.signing_age.apply(lambda x: x.days // 365)

    #convert signing_date to a date instead of a datetime
    df['signing_date'] = pd.DatetimeIndex(df['signing_date']).date

    #remove row that had NaN for signing year (Defenseman Fyodor Tyutin had
    #the contract listed in other rows also so no data is lost
    df.drop(6991, axis=0, inplace=True)

    #Make signing year an int instead of a float
    df.signing_year = df.signing_year.apply(int)

    #Manually entered league salary cap history
    scap = {
            2005: 39000000,
            2006: 44000000,
            2007: 50300000,
            2008: 56700000,
            2009: 56800000,
            2010: 59400000,
            2011: 64300000,
            2012: 60000000,
            2013: 64300000,
            2014: 69000000,
            2015: 71400000,
            2016: 73000000,
            2017: 75000000,
            2018: 79500000,
            2019: 83000000,
           }

    #Add a column for the total salary cap in a contract's year signed
    df['signing_year_cap'] = df['signing_year'].apply(lambda x: scap[x])
    # and a column for the percentage of the cap in the year signed
    df['cap_pct'] = round(100 * df.cap_hit / df.signing_year_cap, 2)

    #convert birthdate to pandas datetime
    df.birthdate = pd.to_datetime(df.birthdate)

    #convert current season and contract_end to single year ints for ease of
    #calculations
    df.season = df.season.apply(lambda x: int(x[:4]))
    df.contract_end = df.contract_end.apply(lambda x: int(x[:4]))

    #Drop some non-standard situations (i.e. suspensions, season-opening IR,
    # salary retention adjustments)
    df.drop(df[df.first_name.map(len) > 15].index, inplace=True)

    #Drop one extremely young player
    df.drop(df[df.ufa_year > 2030].index, inplace = True)

    #Drop where ufa_year is null
    df.drop(df[df.ufa_year.isnull()].index, inplace=True)
    #Turn the remaining ufa years into integers
    df.ufa_year = df.ufa_year.apply(int)

    #Drop any contracts less than league minimum in 2009
    df.drop(df[df.cap_hit < 500000].index, inplace=True)

    #Replace the position names for two players whose positions don't match the rest
    df.replace(['RW', 'LW'], ['Right Wing', 'Left Wing'], inplace=True)

    #drop entry level contracts
    df.drop(df[df.contract_level == 'entry_level'].index, inplace=True)

    #Add a column for whether the contract was signed as UFA or RFA
    df['signing_status'] = df['signing_year'] >= df['ufa_year']
    df['signing_status'] = df['signing_status'].apply(ufa_check)


    return df

#auxillary function to check if contract was signed as UFA or RFA
def ufa_check(x):
    if x == True:
        return 'UFA'
    else:
        return 'RFA'


In [70]:
contracts = clean_data()

In [72]:
contracts = contracts.groupby('contract_id').head(1)

In [75]:
contracts.Player.unique()

array(['Roberto Luongo', 'Ryan Miller', 'Craig Anderson', ...,
       'Parker Milner', 'Dominik Uher', 'Linden Vey'], dtype=object)

In [78]:
len(contracts.Player.unique())

1053

In [95]:
contracts = contracts[contracts.position != 'Goaltender']

In [96]:
len(contracts.Player.unique())

935

In [ ]:
contracts.sort_values()

In [76]:
from collections import defaultdict

In [77]:
t = defaultdict(pd.DataFrame)

In [165]:
allstats = defaultdict(pd.DataFrame)
allsumstats = defaultdict(pd.DataFrame)
allmeanstats = defaultdict(pd.DataFrame)

for player in contracts['Player'].unique():
    allstats[player] = dfsummable[df.Player == player]
    allstats[player].sort_values(by='Season', inplace=True)
    allstats[player]['Season_index'] = pd.date_range(end='2018', 
                                               periods = allstats[player].shape[0], 
                                               freq='Y')
    allstats[player].set_index(allstats[player].Season_index, inplace=True)
    allsumstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=3)
                  .agg(np.sum))
    allmeanstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=3)
              .agg(np.mean))
    allmeanstats[player]['SH%'] = allsumstats[player]['Shots'] / allsumstats[player]['Goals']
    allmeanstats[player]['Faceoffs %'] = allsumstats[player]['Faceoffs Won'] / (
        allsumstats[player]['Faceoffs Won'] + allsumstats[player]['Faceoffs Lost'])

/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [98]:
len(allstats)

935

In [166]:
allstats['Tomas Hertl']

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Season_index
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,2013 Tomas Hertl,Tomas Hertl,2013,C,37,567.150,15,10,5,5,...,0,10,18,18,39,54,13,29,22,2013-12-31
2014-12-31,2014 Tomas Hertl,Tomas Hertl,2014,C,82,1193.300,13,18,6,12,...,0,15,64,29,106,98,33,38,45,2014-12-31
2015-12-31,2015 Tomas Hertl,Tomas Hertl,2015,C,81,1293.820,21,25,12,13,...,0,24,41,35,90,109,60,256,201,2015-12-31
2016-12-31,2016 Tomas Hertl,Tomas Hertl,2016,C,49,843.967,10,12,7,5,...,0,6,18,30,76,60,42,245,229,2016-12-31
2017-12-31,2017 Tomas Hertl,Tomas Hertl,2017,C,79,1430.020,22,24,14,10,...,1,19,45,60,84,118,71,587,569,2017-12-31


In [167]:
allsumstats['Tomas Hertl']

,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,Shots,iCF,iFF,...,Major,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-31,200.0,3054.270,49.0,53.0,23.0,30.0,102.0,445.0,803.0,612.0,...,0.0,0.0,49.0,123.0,82.0,235.0,261.0,106.0,323.0,268.0
2016-12-31,212.0,3331.087,44.0,55.0,25.0,30.0,99.0,447.0,820.0,618.0,...,0.0,0.0,45.0,123.0,94.0,272.0,267.0,135.0,539.0,475.0
2017-12-31,209.0,3567.807,53.0,61.0,33.0,28.0,114.0,477.0,863.0,671.0,...,1.0,1.0,49.0,104.0,125.0,250.0,287.0,173.0,1088.0,999.0


In [168]:
allmeanstats['Tomas Hertl']

,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,Shots,iCF,iFF,...,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,SH%,Faceoffs %
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-31,66.666667,1018.090000,16.333333,17.666667,7.666667,10.000000,34.0,148.333333,267.666667,204.000000,...,16.333333,41.000000,27.333333,78.333333,87.000000,35.333333,107.666667,89.333333,9.081633,0.546531
2016-12-31,70.666667,1110.362333,14.666667,18.333333,8.333333,10.000000,33.0,149.000000,273.333333,206.000000,...,15.000000,41.000000,31.333333,90.666667,89.000000,45.000000,179.666667,158.333333,10.159091,0.531558
2017-12-31,69.666667,1189.269000,17.666667,20.333333,11.000000,9.333333,38.0,159.000000,287.666667,223.666667,...,16.333333,34.666667,41.666667,83.333333,95.666667,57.666667,362.666667,333.000000,9.000000,0.521322


In [142]:
pavelski = allstats['Joe Pavelski']

In [143]:
pavelski

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Season_index
Season_index,,,,,,,,,,,,,,,,,,,,,
2007-12-31,2007 Joe Pavelski,Joe Pavelski,2007,C,82,1157.100,19,21,15,6,...,0,28,26,37,67,92,59,268,232,2007-12-31
2008-12-31,2008 Joe Pavelski,Joe Pavelski,2008,C,80,1517.080,25,34,19,15,...,1,32,41,58,82,106,74,717,557,2008-12-31
2009-12-31,2009 Joe Pavelski,Joe Pavelski,2009,C,67,1305.070,25,26,19,7,...,0,21,38,42,58,90,60,477,344,2009-12-31
2010-12-31,2010 Joe Pavelski,Joe Pavelski,2010,C,74,1454.370,20,46,27,19,...,0,26,49,51,61,104,67,554,466,2010-12-31
2011-12-31,2011 Joe Pavelski,Joe Pavelski,2011,C,82,1690.320,31,30,20,10,...,0,25,61,73,50,108,84,507,357,2011-12-31
2012-12-31,2012 Joe Pavelski,Joe Pavelski,2012,C,48,907.667,16,15,11,4,...,0,13,28,38,48,52,44,342,318,2012-12-31
2013-12-31,2013 Joe Pavelski,Joe Pavelski,2013,C,82,1627.720,41,38,28,10,...,0,26,57,56,47,72,68,675,531,2013-12-31
2014-12-31,2014 Joe Pavelski,Joe Pavelski,2014,C,82,1650.250,37,33,22,11,...,0,29,64,57,97,77,86,642,505,2014-12-31
2015-12-31,2015 Joe Pavelski,Joe Pavelski,2015,C,82,1624.900,38,40,24,16,...,0,18,82,46,147,92,67,516,422,2015-12-31


In [136]:
pavelski.set_index(pavelski.Season_index, inplace=True)

In [154]:
pavsumstat = pavelski[summable_stats].rolling(window=3, min_periods=3).agg([np.sum, np.mean])

In [158]:
pavsumstat.GP['sum']

Season_index
2007-12-31      NaN
2008-12-31      NaN
2009-12-31    229.0
2010-12-31    221.0
2011-12-31    223.0
2012-12-31    204.0
2013-12-31    212.0
2014-12-31    212.0
2015-12-31    246.0
2016-12-31    245.0
2017-12-31    245.0
Name: sum, dtype: float64

In [159]:
pavsumstat

GP                  TOI               Goals             \
                sum       mean       sum         mean    sum       mean   
Season_index                                                              
2007-12-31      NaN        NaN       NaN          NaN    NaN        NaN   
2008-12-31      NaN        NaN       NaN          NaN    NaN        NaN   
2009-12-31    229.0  76.333333  3979.250  1326.416667   69.0  23.000000   
2010-12-31    221.0  73.666667  4276.520  1425.506667   70.0  23.333333   
2011-12-31    223.0  74.333333  4449.760  1483.253333   76.0  25.333333   
2012-12-31    204.0  68.000000  4052.357  1350.785667   67.0  22.333333   
2013-12-31    212.0  70.666667  4225.707  1408.569000   88.0  29.333333   
2014-12-31    212.0  70.666667  4185.637  1395.212333   94.0  31.333333   
2015-12-31    246.0  82.000000  4902.870  1634.290000  116.0  38.666667   
2016-12-31    245.0  81.666667  4823.650  1607.883333  104.0  34.666667   
2017-12-31    245.0  81.666667  4752.350  1584.116667   89.0  29.666667   

             Total Assists            First Assists                ...      \
                       sum       mean           sum       mean     ...       
Season_index                                                       ...       
2007-12-31             NaN        NaN           NaN        NaN     ...       
2008-12-31             NaN        NaN           NaN        NaN     ...       
2009-12-31            81.0  27.000000          53.0  17.666667     ...       
2010-12-31           106.0  35.333333          65.0  21.666667     ...       
2011-12-31           102.0  34.000000          66.0  22.000000     ...       
2012-12-31            91.0  30.333333          58.0  19.333333     ...       
2013-12-31            83.0  27.666667          59.0  19.666667     ...       
2014-12-31            86.0  28.666667          61.0  20.333333     ...       
2015-12-31           111.0  37.000000          74.0  24.666667     ...       
2016-12-31           112.0  37.333333          69.0  23.000000     ...       
2017-12-31           123.0  41.000000          73.0  24.333333     ...       

               Hits             Hits Taken             Shots Blocked  \
                sum        mean        sum        mean           sum   
Season_index                                                           
2007-12-31      NaN         NaN        NaN         NaN           NaN   
2008-12-31      NaN         NaN        NaN         NaN           NaN   
2009-12-31    207.0   69.000000      288.0   96.000000         193.0   
2010-12-31    201.0   67.000000      300.0  100.000000         201.0   
2011-12-31    169.0   56.333333      302.0  100.666667         211.0   
2012-12-31    159.0   53.000000      264.0   88.000000         195.0   
2013-12-31    145.0   48.333333      232.0   77.333333         196.0   
2014-12-31    192.0   64.000000      201.0   67.000000         198.0   
2015-12-31    291.0   97.000000      241.0   80.333333         221.0   
2016-12-31    374.0  124.666667      239.0   79.666667         226.0   
2017-12-31    370.0  123.333333      241.0   80.333333         210.0   

                        Faceoffs Won             Faceoffs Lost              
                   mean          sum        mean           sum        mean  
Season_index                                                                
2007-12-31          NaN          NaN         NaN           NaN         NaN  
2008-12-31          NaN          NaN         NaN           NaN         NaN  
2009-12-31    64.333333       1462.0  487.333333        1133.0  377.666667  
2010-12-31    67.000000       1748.0  582.666667        1367.0  455.666667  
2011-12-31    70.333333       1538.0  512.666667        1167.0  389.000000  
2012-12-31    65.000000       1403.0  467.666667        1141.0  380.333333  
2013-12-31    65.333333       1524.0  508.000000        1206.0  402.000000  
2014-12-31    66.000000       1659.0  553.000000        1354.0  451.333333  
2015-12-31    73.

ValueError: Wrong number of items passed 2, placement implies 1

In [45]:
class ComputeNearestMean(BaseEstimator, TransformerMixin):
    """Compute a mean price for similar vehicles.
    """
    def __init__(self, window=5):
        self.window = window

    def get_params(self, **kwargs):
        return {'window': self.window}

    def fit(self, X, y):
        X = X.sort_values(by=['saledate_converted'])
        g = X.groupby('ModelID')['SalePrice']
        m = g.apply(lambda x: x.rolling(self.window).agg([np.mean]))

        ids = X[['saledate_converted', 'ModelID', 'SalesID']]
        z = pd.concat([m, ids], axis=1)
        z['saledate_converted'] = z.saledate_converted + timedelta(1)

        # Some days will have more than 1 transaction for a particular model,
        # take the last mean (which has most info)
        z = z.drop('SalesID', axis=1)
        groups = ['ModelID', 'saledate_converted']
        self.averages = z.groupby(groups).apply(lambda x: x.tail(1))

        # This is kinda unsatisfactory, but at least ensures
        # we can always make predictions
        self.default_mean = X.SalePrice.mean()
        return self

    def transform(self, X):
        near_price = pd.merge(self.averages, X, how='outer',
                              on=['ModelID', 'saledate_converted'])
        nxcols = ['ModelID', 'saledate_converted']
        near_price = near_price.set_index(nxcols).sort_index()
        g = near_price['mean'].groupby(level=0)
        filled_means = g.transform(lambda x: x.fillna(method='ffill'))
        near_price['filled_mean_price'] = filled_means
        near_price = near_price[near_price['SalesID'].notnull()]
        missing_mean = near_price.filled_mean_price.isnull()
        near_price['no_recent_transactions'] = missing_mean
        near_price['filled_mean_price'].fillna(self.default_mean, inplace=True)
        return near_price


NameError: name 'BaseEstimator' is not defined